# f) Ngonye Falls Power and Energy Periods
Produces summary, agrregated results from the daily power and energy modelling.


## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| [Model Name]_pe_daily.csv | Notebook: e_power_energy | Daily power output  |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
|    [Model Name]_pe_daily.csv ||
|    [Model Name]_pe_monthly.csv||
|    [Model Name]_pe_yearly.csv||
|    [Model Name]_pe_percentiles.csv||
|    [Model Name]_pe_monthly_day.csv||
|    [Model Name]_pe_daily_slim.csv||
|    [Model Name]_pe_weekly.csv||
|    [Model Name]_pe_weekly_slim.csv||
|    [Model Name]_pe_calmonthly.csv||
|    [Model Name]_pe_fdc.csv||
|    [Model Name]_pe_power_exceed.csv||
|    models.csv| |

## Libraries and Setup

In [5]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math
import statsmodels.tsa.stattools as statools

## Parameters

In [6]:
input_data='./input_data/'
#output_data='./output_data/2020/'
output_data='./output_data/'

In [239]:
model_name='EWRAllCs'#'1 Unit Available' MW153#MW144 'MW162'#'Fixed Tailwater'#'Base Case' Headpond50cm EWRAllCs
dryrun=False #Dont write output files

In [240]:


models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
#models=pd.read_csv(input_data + "2020/models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="/scenarios/" + out_prefix

category_set_name=model['EWRCatSet']#'Recommendation 1'
headpond_lift=model['HeadpondLift']#0
plant_capacity=model['Capacity'] #Base 180MW nominal capacity. Only used to calculate 'nominal capacity factor'
units=4 #Design no of units
units_avialable=model['UnitsAvailable'] #Available no of units - to see what happens during testing/commissioning

rated_flow_unit=model['RatedDischarge'] #250
max_flow_unit=rated_flow_unit*model['MaximumLoadFactor'] #275
canal_capacity=max_flow_unit*4 #canal capacity 1100 cumec 
min_flow_unit=model['MinimumFlowUnit'] #50cumec minimum flow for a turbine
fixed_tailwater_level=model['FixedTailwater']#False #970 
head_minimum=model['MinimumHead'] #minimum head 10m


model

OutputPrefix                                                            allcs
Description                 All river reaches at a C rather than B environ...
EWRCatSet                                                              All Cs
HeadpondLift                                                                0
Capacity                                                                  180
MinimumHead                                                                10
MinimumFlowUnit                                                            50
FixedTailwater                                                              0
UnitsAvailable                                                              4
RatedDischarge                                                            250
RatedHead                                                                21.7
RatedTurbineEfficiency                                                  0.943
RatedGeneratorEfficiency                                        

## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [241]:
daily=pd.read_csv(output_data + out_prefix + '_pe_daily.csv') #ngonye_daily_WC1_1_base_pe
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
daily.tail(5)

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,...,EffGen3,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,ExceedanceRnd
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-26,2017-10-07,204.0855,0.982689,200.552607,0.955,2017,9,26,2017.09,2016,...,NaN,NaN,27.704817,NaN,NaN,NaN,27.704817,664.915604,0.138142,0.955
2017-09-27,2017-10-08,204.0855,0.982689,200.552607,0.955,2017,9,27,2017.09,2016,...,NaN,NaN,27.704817,NaN,NaN,NaN,27.704817,664.915604,0.138142,0.955
2017-09-28,2017-10-09,204.0855,0.982689,200.552607,0.955,2017,9,28,2017.09,2016,...,NaN,NaN,27.704817,NaN,NaN,NaN,27.704817,664.915604,0.138142,0.955
2017-09-29,2017-10-10,204.0855,0.982689,200.552607,0.955,2017,9,29,2017.09,2016,...,NaN,NaN,27.704817,NaN,NaN,NaN,27.704817,664.915604,0.138142,0.955
2017-09-30,2017-10-11,200.9197,0.995488,200.013130,0.958,2017,9,30,2017.09,2016,...,NaN,NaN,27.588852,NaN,NaN,NaN,27.588852,662.132456,0.137935,0.960


In [242]:
daily=daily.drop(daily.loc['2020-05-01':].index)
daily.tail(5)

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,...,EffGen3,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,ExceedanceRnd
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-26,2017-10-07,204.0855,0.982689,200.552607,0.955,2017,9,26,2017.09,2016,...,NaN,NaN,27.704817,NaN,NaN,NaN,27.704817,664.915604,0.138142,0.955
2017-09-27,2017-10-08,204.0855,0.982689,200.552607,0.955,2017,9,27,2017.09,2016,...,NaN,NaN,27.704817,NaN,NaN,NaN,27.704817,664.915604,0.138142,0.955
2017-09-28,2017-10-09,204.0855,0.982689,200.552607,0.955,2017,9,28,2017.09,2016,...,NaN,NaN,27.704817,NaN,NaN,NaN,27.704817,664.915604,0.138142,0.955
2017-09-29,2017-10-10,204.0855,0.982689,200.552607,0.955,2017,9,29,2017.09,2016,...,NaN,NaN,27.704817,NaN,NaN,NaN,27.704817,664.915604,0.138142,0.955
2017-09-30,2017-10-11,200.9197,0.995488,200.013130,0.958,2017,9,30,2017.09,2016,...,NaN,NaN,27.588852,NaN,NaN,NaN,27.588852,662.132456,0.137935,0.960


In [243]:
daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 33968 entries, 1924-10-01 to 2017-09-30
Data columns (total 80 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   LaggedDate                  33968 non-null  object 
 1   VicFalls                    33968 non-null  float64
 2   Conversion                  33968 non-null  float64
 3   Flow                        33968 non-null  float64
 4   Exceedance                  33968 non-null  float64
 5   Year                        33968 non-null  int64  
 6   Month                       33968 non-null  int64  
 7   Day                         33968 non-null  int64  
 8   MonthId                     33968 non-null  float64
 9   WaterYear                   33968 non-null  int64  
 10  WaterMonth                  33968 non-null  int64  
 11  WaterDay                    33968 non-null  int64  
 12  WaterWeek                   33968 non-null  int64  
 13  Volume        

In [244]:
#daily['Turbine1FlowCurtailed']=np.where((daily['Turbines']<2) & (daily['FlowTurbine1']<daily['FlowTurbine1'].max()),True,False)
#daily['Turbine1HeadCurtailed']=np.where((daily['FlowTurbine1']>=daily['FlowTurbine1'].max()) & (daily['PowerTurbine1']<daily['PowerTurbine1'].max()),True,False)
#daily['Turbine1HeadCurtailed2']=np.where((daily['FlowCanal']>=daily['FlowTurbine1'].max()) & (daily['HeadTurbine1']<daily['HeadTurbine1'].max()),True,False)

## Monthly


In [245]:

monthly=daily.groupby("MonthId").mean().drop(['Day','EWRBandNo','EWRRefExceedance','Energy'],axis=1)

if 'VicFalls' in monthly.columns:
    monthly=monthly.drop(['VicFalls','Conversion','Volume','WaterDay','WaterWeek','Volume'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),    
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(plant_capacity*24*monthly['Days'])

monthly.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1116 entries, 1924.1 to 2017.09
Data columns (total 75 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Flow                        1116 non-null   float64
 1   Exceedance                  1116 non-null   float64
 2   Year                        1116 non-null   float64
 3   Month                       1116 non-null   float64
 4   WaterYear                   1116 non-null   float64
 5   WaterMonth                  1116 non-null   float64
 6   Flow_difference             1116 non-null   float64
 7   Flow_difference_pct         1116 non-null   float64
 8   EWRChannelA                 1116 non-null   float64
 9   EWRChannelC                 1116 non-null   float64
 10  EWRChannelD                 1116 non-null   float64
 11  EWRChannelE                 1116 non-null   float64
 12  EWRChannelFG                1116 non-null   float64
 13  EWRTotal               

## Yearly

In [246]:
yearly=daily.groupby("WaterYear").agg(
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),   
   PowerTurbine1_max=('PowerTurbine1', 'max'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(plant_capacity*24*365)


In [247]:
yearly=yearly.drop(yearly.loc['2019':].index)
yearly.tail(5)

,Flow_max,Flow_min,EWRProportion_max,EWRProportion_min,HeadTurbine1_max,HeadTurbine1_min,PowerTurbine1_max,LevelTailwater_max,LevelTailwater_min,Power_max,Power_min,Energy,CapFactor
WaterYear,,,,,,,,,,,,,
2012,3614.802471,223.273423,0.438768,0.047608,25.050164,13.168827,59.694508,974.894271,964.716509,191.796153,28.672622,868249.625702,0.550640
2013,3574.946119,195.061590,0.410127,0.049336,25.408807,13.247224,59.622805,974.815874,964.405522,191.583043,25.037703,952594.714528,0.604132
2014,1460.253290,208.780681,0.472549,0.083619,25.063781,18.660100,59.604012,969.484512,964.706890,194.111267,29.469088,915406.360059,0.580547
2015,2975.524395,176.218173,0.429317,0.058882,25.363761,14.500960,60.983038,973.562138,964.405522,191.481856,25.679180,810655.966836,0.514115
2016,3889.991232,156.550907,0.427024,0.041480,25.365880,12.642403,60.983038,975.420695,964.405522,192.151618,21.800102,875516.514997,0.555249


In [248]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))


## Power and energy exceedance

In [249]:
power_exceed=pd.DataFrame(index=np.arange(0,1.05,0.05))
power_exceed.index=power_exceed.index.rename('Exceedance')
power_exceed['Power']=np.percentile(daily['Power'],(1-power_exceed.index)*100)
power_exceed.head(3)

,Power
Exceedance,
0.00,195.059917
0.05,183.678165
0.10,175.024562


In [250]:
edc=pd.DataFrame(index=np.arange(0,1.05,0.05))
edc['Energy']=np.percentile(yearly['Energy'],(1-edc.index)*100)

edc.head(3)

,Energy
0.00,1.048285e+06
0.05,9.867600e+05
0.10,9.507935e+05


In [251]:
yearly['Exceedance']=pd.merge_asof(yearly.reset_index().sort_values('Energy'),edc.reset_index().sort_values('Energy'),left_on='Energy',right_on='Energy').set_index('WaterYear')['index']

## Calendar years
Jan-Dec rather than Oct-Sept

In [252]:
min_year=daily['Year'].min()
max_year=daily['Year'].max()


In [253]:
calyearly=daily.groupby("Year").agg(  
   Energy=('Energy', 'sum'),
).drop([min_year,max_year],axis=0)
calyearly['CapFactor']=calyearly['Energy']/(plant_capacity*24*365)
calyearly.head(5)

,Energy,CapFactor
Year,,
1925,877100.201800,0.556253
1926,805231.728339,0.510675
1927,879496.346554,0.557773
1928,856712.729914,0.543324
1929,783902.902576,0.497148


## Monthly Day
15th of every month

In [254]:
monthly_day=daily.loc[daily['Day']==15]

## Daily Slim
Slim has less columns and values are rounded so file size is reduced

In [255]:
daily_slim=daily.loc[:,
                     ['Year',
                      'Month',
                      'Day',
                      'WaterYear',
                      'Flow',
                      'EWRRefExceedance',
                      'EWRBandNo',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'Turbines',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',    
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion': 2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,    
                    })
daily_slim.tail(12)

,Year,Month,Day,WaterYear,Flow,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,EffTurbine1,EffTurbine2,EffTurbine3,EffTurbine4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-19,2017,9,19,2016,216,0.80,4,20,4,3,...,0.91,0.0,0.0,0.0,31.1,0.0,0.0,0.0,31.1,746
2017-09-20,2017,9,20,2016,213,0.80,4,20,4,3,...,0.91,0.0,0.0,0.0,30.5,0.0,0.0,0.0,30.5,731
2017-09-21,2017,9,21,2016,213,0.82,4,20,4,3,...,0.91,0.0,0.0,0.0,30.5,0.0,0.0,0.0,30.5,731
2017-09-22,2017,9,22,2016,209,0.82,4,20,4,3,...,0.91,0.0,0.0,0.0,29.7,0.0,0.0,0.0,29.7,711
2017-09-23,2017,9,23,2016,203,0.82,4,20,4,3,...,0.91,0.0,0.0,0.0,28.2,0.0,0.0,0.0,28.2,677
2017-09-24,2017,9,24,2016,200,0.82,4,20,4,3,...,0.91,0.0,0.0,0.0,27.7,0.0,0.0,0.0,27.7,664
2017-09-25,2017,9,25,2016,200,0.82,4,20,4,3,...,0.91,0.0,0.0,0.0,27.7,0.0,0.0,0.0,27.7,664
2017-09-26,2017,9,26,2016,200,0.82,4,20,4,3,...,0.91,0.0,0.0,0.0,27.7,0.0,0.0,0.0,27.7,664
2017-09-27,2017,9,27,2016,200,0.82,4,20,4,3,...,0.91,0.0,0.0,0.0,27.7,0.0,0.0,0.0,27.7,664


## Weekly & Weekly Slim
Slim has less columns and values are rounded so file size is reduced

In [256]:
weekly=daily.groupby(["WaterYear","WaterWeek"]).mean().drop(['Month','WaterMonth','Year','MonthId','Day','WaterDay','EWRBandNo','EWRRefExceedance','Turbines','Energy'],axis=1)

#if 'VicFalls' in weekly.columns:
weekly=weekly.drop(['VicFalls','Conversion','Volume'],axis=1)

weekly=weekly.join(daily.groupby(["WaterYear","WaterWeek"]).agg(    
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
))
weekly=weekly.join(daily[["WaterYear","WaterWeek","Flow"]].groupby(["WaterYear","WaterWeek"]).count().rename(columns={'Flow':'Count'}))
weekly['CapFactor']=weekly['Energy']/(plant_capacity*24*weekly['Count'])
weekly=weekly.join(daily.reset_index('Date')[["WaterYear","WaterWeek","Date"]].groupby(["WaterYear","WaterWeek"]).first().rename(columns={'Date':'StartDate'}))

weekly.tail(12)

Flow  Exceedance  Flow_difference  \
WaterYear WaterWeek                                            
2016      42         451.943382    0.593143         6.536288   
          43         411.172524    0.638714         4.724796   
          44         377.834520    0.678857         5.261728   
          45         344.446201    0.716714         4.257422   
          46         322.314771    0.751857         2.882230   
          47         296.218764    0.791143         3.371720   
          48         271.256811    0.834000         3.832214   
          49         250.464258    0.873286         3.094980   
          50         226.821410    0.915571         2.855981   
          51         215.657828    0.933429         1.480060   
          52         200.905599    0.953571         1.303758   
          53         200.013130    0.958000         0.539477   

                     Flow_difference_pct  EWRChannelA  EWRChannelC  \
WaterYear WaterWeek                                                  
2016      42                    0.014538    35.000000     7.000000   
          43                    0.011456    35.000000     7.000000   
          44                    0.013976    29.285714     5.857143   
          45                    0.012366    25.000000     5.000000   
          46                    0.009020    25.000000     5.000000   
          47                    0.011315    25.000000     5.000000   
          48                    0.014055    24.285714     4.857143   
          49                    0.012470    20.000000     4.000000   
          50                    0.012523    20.000000     4.000000   
          51                    0.006946    20.000000     4.000000   
          52                    0.006443    20.000000     4.000000   
          53                    0.002697    20.000000     4.000000   

                     EWRChannelD  EWRChannelE  EWRChannelFG    EWRTotal  ...  \
WaterYear WaterWeek                                                      ...   
2016      42            6.000000     7.000000    108.000000  163.000000  ...   
          43            6.000000     7.000000    108.000000  163.000000  ...   
          44            4.857143     4.714286     80.571429  125.285714  ...   
          45            4.000000     3.000000     60.000000   97.000000  ...   
          46            4.000000     3.000000     60.000000   97.000000  ...   
          47            4.000000     3.000000     60.000000   97.000000  ...   
          48            3.857143     2.857143     57.714286   93.571429  ...   
          49            3.000000     2.000000     44.000000   73.000000  ...   
          50            3.000000     2.000000     44.000000   73.000000  ...   
          51            3.000000     2.000000     44.000000   73.000000  ...   
          52            3.000000     2.000000     44.000000   73.000000  ...   
          53            3.000000     2.000000     44.000000   73.000000  ...   

                     HeadTurbine1_max  HeadTurbine1_min  LevelTailwater_max  \
WaterYear WaterWeek                                                           
2016      42                24.249516         23.914820          965.589031   
          43                24.190069         23.957654          965.380858   
          44                24.510100         24.190758          965.236958   
          45                24.458612         24.288474          965.083218   
          46                24.604265         24.474430          964.993439   
          47                24.758998         24.650284          964.908128   
          48                24.898172         24.788919          964.840915   
          49                24.950451         24.854810          964.778028   
          50                25.029629         24.973519          964.730913   
          51                25.061363         25.029629          964.713474   
          52                25.083250         25.077923          964.706442   
 

In [257]:
weekly_slim=weekly.loc[:,
                     [
                      'StartDate',
                      'Flow',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion':2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,
                    })

## Calendar Months

In [258]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly=calmonthly.join(monthly.groupby(["WaterMonth"]).agg(    
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_mean=('EWRProportion', 'mean'),
   EWRProportion_median=('EWRProportion', 'median'),
   EWRProportion_min=('EWRProportion', 'min'),
    
    EWRTotal_max=('EWRTotal', 'max'),
   EWRTotal_mean=('EWRTotal', 'mean'),
   EWRTotal_median=('EWRTotal', 'median'),
   EWRTotal_min=('EWRTotal', 'min'),   
    
   FlowCanal_max=('FlowCanal', 'max'),
   FlowCanal_mean=('FlowCanal', 'mean'),
   FlowCanal_median=('FlowCanal', 'median'),
   FlowCanal_min=('FlowCanal', 'min'),     
    
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_mean=('HeadTurbine1', 'mean'),
   HeadTurbine1_median=('HeadTurbine1', 'median'),
   HeadTurbine1_min=('HeadTurbine1', 'min'), 

   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_mean=('LevelTailwater', 'mean'),    
   LevelTailwater_median=('LevelTailwater', 'median'),    
   LevelTailwater_min=('LevelTailwater', 'min'),  
   LevelHeadpond_max=('LevelHeadpond', 'max'),
   LevelHeadpond_mean=('LevelHeadpond', 'mean'),    
   LevelHeadpond_median=('LevelHeadpond', 'median'),    
   LevelHeadpond_min=('LevelHeadpond', 'min'),  
   Turbines_mean=('Turbines', 'mean'),      
   Power_max=('Power', 'max'),
   Power_mean=('Power', 'mean'),
   Power_median=('Power', 'median'),
   Power_min=('Power', 'min'),    
   Energy_max=('Energy', 'max'),
   Energy_mean=('Energy', 'mean'),
   Energy_median=('Energy', 'median'),    
   Energy_min=('Energy', 'min'),
))
calmonthly['Energy_P95']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.05)
calmonthly['Energy_P90']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.10)
calmonthly['Energy_P75']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.25)
calmonthly['Energy_P25']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.75)
calmonthly['Energy_P10']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.90)
calmonthly['Energy_P5']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.95)

calmonthly['Energy_stdev']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').std()
calmonthly['Energy_coefvar']=calmonthly['Energy_stdev']/calmonthly['Energy_mean']

calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,Energy_median,Energy_min,Energy_P95,Energy_P90,Energy_P75,Energy_P25,Energy_P10,Energy_P5,Energy_stdev,Energy_coefvar
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.466357,0.321297,0.324670,0.222229,97.0,75.856746,80.000000,41.0,...,23906.593232,8023.239094,18895.552109,20912.531120,21716.545238,30281.857477,35160.960076,40464.665674,7459.418165,0.282017
2,Nov,11,0.325346,0.266672,0.268798,0.176499,92.0,71.885305,75.000000,40.0,...,30048.843147,17599.159331,21581.363313,23703.594353,25729.837944,35808.157755,44322.336152,48892.452977,9325.491627,0.290641
3,Dec,12,0.235862,0.195610,0.196144,0.129025,103.0,81.322581,85.000000,47.0,...,50835.279392,30044.425073,38901.114834,40580.787569,45483.169785,64270.389545,72065.306182,87906.276576,14768.524861,0.266591
4,Jan,1,0.182821,0.147300,0.150938,0.080737,124.0,97.903226,101.000000,60.0,...,80070.941305,43995.197079,58890.018721,64737.634300,72291.951080,100983.084623,119503.568548,123701.498053,20455.212254,0.235942
5,Feb,2,0.157730,0.108892,0.116230,0.020434,168.0,117.355183,116.000000,71.0,...,101202.513765,4972.912035,73215.847960,81299.225486,90727.907529,113078.593762,121150.563523,123172.556851,18662.905838,0.186539
6,Mar,3,0.148788,0.081940,0.080351,0.035518,278.0,169.274714,161.322581,87.0,...,112004.701194,0.000000,20050.327214,78385.740768,95755.855911,122230.294314,131714.309324,134732.403458,30710.308215,0.293552
7,Apr,4,0.134881,0.072809,0.070702,0.042988,313.0,199.655914,191.000000,102.0,...,100044.115710,0.000000,58607.459908,73218.225413,87273.472395,114982.549395,130638.525216,134907.835457,26547.673910,0.269158
8,May,5,0.189754,0.116985,0.116323,0.059363,304.0,238.554977,253.000000,125.0,...,116877.993126,44299.068846,88481.688970,93379.529789,106016.365248,126568.221168,132189.568362,135020.224699,16388.456581,0.143042
9,Jun,6,0.304233,0.173488,0.171731,0.096485,260.0,186.157706,185.000000,98.0,...,116311.431502,36045.448107,53849.445445,75186.771472,98660.704650,122833.155752,124868.900235,125705.574694,22522.209378,0.210059


## Flow duration curve

In [259]:
daily['ExceedanceRnd']=np.round(daily['Exceedance']*2,2)/2
fdc=daily.reset_index().groupby('ExceedanceRnd').mean().drop(['VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)
fdc

,Flow,Exceedance,Volume,Flow_difference,Flow_difference_pct,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,EffGen2,EffGen3,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec
ExceedanceRnd,,,,,,,,,,,,,,,,,,,,,
0.000,8865.175953,0.001412,0.765951,137.615867,0.015867,0.026324,1.102941,44.691176,7.970588,8.779412,...,0.98,0.98,0.98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.005,6478.790089,0.004989,0.559767,115.269934,0.017456,0.041149,1.091954,50.563218,9.195402,9.896552,...,0.98,0.98,0.98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.010,5544.712707,0.010047,0.479063,68.659047,0.012349,0.077041,1.289941,51.278107,9.218935,9.769231,...,0.98,0.98,0.98,9.617783,9.617783,9.617783,9.617783,38.471131,923.307133,0.007106
0.015,5190.964012,0.014953,0.448499,88.798283,0.017178,0.105349,1.401163,49.284884,8.767442,9.238372,...,0.98,0.98,0.98,22.887085,22.887085,22.887085,22.887085,91.548338,2197.160113,0.017651
0.020,4777.593171,0.020076,0.412784,113.350030,0.023656,0.110465,1.406977,49.162791,8.750000,9.395349,...,0.98,0.98,0.98,24.716775,24.716775,24.716775,24.716775,98.867101,2372.810412,0.020720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.980,185.643897,0.979851,0.016040,2.397496,0.012948,0.897619,4.511905,16.904762,2.976190,2.184524,...,NaN,NaN,NaN,29.431413,NaN,NaN,NaN,29.431413,706.353915,0.158502
0.985,175.260665,0.985178,0.015143,1.918049,0.010903,0.920385,4.682692,16.322115,2.894231,2.139423,...,NaN,NaN,NaN,27.749495,NaN,NaN,NaN,27.749495,665.987888,0.158381
0.990,165.025295,0.990459,0.014258,2.091723,0.012641,0.944222,4.859259,15.733333,2.888889,2.125926,...,NaN,NaN,NaN,26.167054,NaN,NaN,NaN,26.167054,628.009289,0.158550


## Add overall summary data to the model file

In [260]:

models.at[model_name,'Mean']=yearly['Energy'].mean()
models.at[model_name,'Max']=yearly['Energy'].max()
models.at[model_name,'Min']=yearly['Energy'].min()

models.at[model_name,'P95']=yearly['Energy'].quantile(0.05)
models.at[model_name,'P90']=yearly['Energy'].quantile(0.10)
models.at[model_name,'P75']=yearly['Energy'].quantile(0.25)
models.at[model_name,'P50']=yearly['Energy'].quantile(0.5)
models.at[model_name,'P25']=yearly['Energy'].quantile(0.75)
models.at[model_name,'P10']=yearly['Energy'].quantile(0.90)
models.at[model_name,'P5']=yearly['Energy'].quantile(0.95)

models.at[model_name,'StDev']=yearly['Energy'].std()
models.at[model_name,'CoefVar']=models.at[model_name,'StDev']/models.at[model_name,'Mean']

models.at[model_name,'CapFactor']=yearly['CapFactor'].mean()

models.at[model_name,'CalYearMean']=calyearly['Energy'].mean()
models.at[model_name,'CalYearStDev']=calyearly['Energy'].std()
models.at[model_name,'CalYearCoefVar']=models.at[model_name,'CalYearStDev']/models.at[model_name,'CalYearMean']

models.at[model_name,'Power_max']=yearly['Power_max'].max()
models.at[model_name,'Power_Turbine1_max']=yearly['PowerTurbine1_max'].max()

models.at[model_name,'CapFactor_Power_max']=calyearly['Energy'].mean()/(yearly['Power_max'].max()*24*365)

rated_power_unit=rated_flow_unit*model['RatedHead']*model['RatedTurbineEfficiency']*model['RatedGeneratorEfficiency']*9.81/1000
models.at[model_name,'RatedPower']=rated_power_unit

models.loc[model_name]



OutputPrefix                                                            allcs
Description                 All river reaches at a C rather than B environ...
EWRCatSet                                                              All Cs
HeadpondLift                                                                0
Capacity                                                                  180
MinimumHead                                                                10
MinimumFlowUnit                                                            50
FixedTailwater                                                              0
UnitsAvailable                                                              4
RatedDischarge                                                            250
RatedHead                                                                21.7
RatedTurbineEfficiency                                                  0.943
RatedGeneratorEfficiency                                        

In [261]:
autocorrel=pd.DataFrame(statools.acf(yearly['Energy'],fft=True,qstat=False,alpha=0.05))
autocorrel

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1,0.493545,0.371898,0.299003,0.278181,0.197108,0.15744,0.111387,0.189689,0.114064,...,-0.181939,-0.0829327,-0.0237096,0.0536704,0.0258465,-0.0305963,-0.0147312,0.0256218,0.00512974,0.0503842
1,"[1.0, 1.0]","[0.29030663058601813, 0.6967842256618993]","[0.12404892917203081, 0.619747344629156]","[0.029086351935500687, 0.5689202198349655]","[-0.005086988867561926, 0.5614495432390361]","[-0.09722831874642826, 0.4914441773475662]","[-0.14229867018428108, 0.4571792048871235]","[-0.19174887882018912, 0.4145222516905731]","[-0.115132190010373, 0.49451078000726467]","[-0.1955947040012072, 0.42372326185413894]",...,"[-0.5718857170806881, 0.20800696499838423]","[-0.47636977655132906, 0.31050446157562517]","[-0.4178681576069295, 0.37044893015222446]","[-0.34054702883114735, 0.44788787072777864]","[-0.3686726345823223, 0.4203656732102297]","[-0.42518541768156354, 0.36399276532869734]","[-0.4094182571618053, 0.3799558927835404]","[-0.36908797671108606, 0.42033159385148067]","[-0.38964873674252704, 0.39990822128508535]","[-0.3443970368507224, 0.4451654277237185]"


In [262]:
lowheadshutdowns=daily.loc[daily['LowHeadShutoff']==1].groupby('WaterYear').count()['Flow'].to_frame().rename(columns={"Flow":"Days"})

lowheadshutdowns['StartDate']=daily[['WaterYear','Flow']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').min()['Date']
lowheadshutdowns['EndDate']=daily[['WaterYear','Flow']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').max()['Date']
lowheadshutdowns['MonthId']=daily[['WaterYear','MonthId']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').min()['MonthId']

#daily.loc[daily['LowHeadShutoff']==1].groupby('WaterYear').min()#['Date']
lowheadshutdowns['Month_power_max']=monthly.loc[lowheadshutdowns['MonthId']][['WaterYear','Power_max']].set_index('WaterYear')
lowheadshutdowns

,Days,StartDate,EndDate,MonthId,Month_power_max
WaterYear,,,,,
1933,3,1934-03-26,1934-03-28,1934.03,181.341984
1947,31,1948-03-04,1948-04-03,1948.03,106.174038
1951,19,1952-03-02,1952-03-20,1952.03,104.964243
1952,9,1953-03-21,1953-03-29,1953.03,187.770124
1955,12,1956-05-05,1956-05-16,1956.05,115.164735
1956,27,1957-03-02,1957-03-28,1957.03,105.902482
1957,60,1958-02-03,1958-04-03,1958.02,110.164461
1960,22,1961-04-02,1961-04-23,1961.04,100.599028
1961,6,1962-04-05,1962-04-12,1962.04,104.188853


In [263]:
lowflowshutdowns=daily.loc[daily['LowFlowShutoff']==1].groupby('Year').count()['Flow'].to_frame().rename(columns={"Flow":"Days"})

lowflowshutdowns['StartDate']=daily[['Year','Flow']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').min()['Date']
lowflowshutdowns['EndDate']=daily[['Year','Flow']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').max()['Date']
lowflowshutdowns['MonthId']=daily[['Year','MonthId']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').min()['MonthId']


lowflowshutdowns['Month_power_max']=monthly.loc[lowflowshutdowns['MonthId']][['Year','Power_max']].set_index('Year')
lowflowshutdowns

,Days,StartDate,EndDate,MonthId,Month_power_max
Year,,,,,
1924,10,1924-10-01,1924-10-10,1924.1,17.827247


## Save Files

In [264]:
#outp=output_data+out_prefix
#daily.to_csv(outp+'_pe_daily.csv')

In [265]:
outp=output_data+out_prefix

if dryrun==False:
    daily.to_csv(outp+'_pe_daily.csv')
    monthly.to_csv(outp+'_pe_monthly.csv')
    yearly.to_csv(outp+'_pe_yearly.csv')
    edc.to_csv(outp+'_pe_percentiles.csv')
    monthly_day.to_csv(outp+'_pe_monthly_day.csv')
    daily_slim.to_csv(outp+'_pe_daily_slim.csv')
    weekly.to_csv(outp+'_pe_weekly.csv')
    weekly_slim.to_csv(outp+'_pe_weekly_slim.csv')
    calmonthly.to_csv(outp+'_pe_calmonthly.csv')
    power_exceed.to_csv(outp+'_pe_power_exceed.csv')
    fdc.to_csv(outp+'_pe_fdc.csv')
    autocorrel.to_csv(outp+'_pe_autocorrel.csv')
    #models.to_csv(input_data + "models.csv")
    models.to_csv(input_data + "models.csv")
    lowheadshutdowns.to_csv(outp + "lowheadshutdowns.csv")
    lowflowshutdowns.to_csv(outp + "lowflowshutdowns.csv")

In [266]:
years=daily['WaterYear'].unique().tolist()

for year in years:
    days=daily.loc[daily.WaterYear==year]
#    days.to_csv(output_data + 'years/' + out_prefix + '_pe_daily_' + str(year) + '.csv')
 